The SA and MA EPR involves an integral over a region where E(\theta) but also the region we're integrating over also depends on \theta.

This means the derivative of this integral is not simply the integral of the derivative, but instead we must use a multidimensional analogue of the Liebniz Rule (https://en.wikipedia.org/wiki/Leibniz_integral_rule) known as the Reynolds Transport Theorem.

\begin{align}
\frac{d\mathcal{L}}{d\theta} &= \frac{\partial \mathcal{L}}{\partial f} \frac{df}{d\theta} + \frac{\partial \mathcal{L}}{\partial \Omega} \frac{d\Omega}{d\theta} \\
 &= \int_{\Omega(\theta)} \frac{d}{d\theta} f(\mathbf{r}, \theta) d\mathbf{r} + \int_{\partial\Omega} v(r)f(\mathbf{r}, \theta) d\mathbf{r}. \label
\end{align}

In [6]:
import numpy as np

mesh_spacing = 0.001   # master knob: grid spacing for area integral
d_theta      = 1e-3   # step for finite-difference check

# field and theta-derivative
def f(x,y,t):  return np.sin(x+t)*np.cos(y-2*t)
def dft(x,y,t): return np.cos(x+t)*np.cos(y-2*t) + 2*np.sin(x+t)*np.sin(y-2*t)

def area_int(g, theta, cx, cy, R):
    m = mesh_spacing
    xmin, xmax = cx - R - m, cx + R + m
    ymin, ymax = cy - R - m, cy + R + m
    xs = np.arange(xmin, xmax + m, m); ys = np.arange(ymin, ymax + m, m)
    xx, yy = np.meshgrid(xs, ys)
    mask = ((xx-cx)**2 + (yy-cy)**2 <= R**2)
    return np.sum(g(xx,yy,theta)[mask]) * (m*m)

def bnd_int(theta, cx, cy, R, vn):
    # sample roughly one point per mesh cell along the circumference
    k = max(64, int(np.ceil(2*np.pi*R / mesh_spacing)))
    phi = np.linspace(0, 2*np.pi, k, endpoint=False)
    xb, yb = cx + R*np.cos(phi), cy + R*np.sin(phi)
    ds = (2*np.pi*R) / k
    return np.sum(vn(phi) * f(xb,yb,theta)) * ds

def L(theta, mode, R0=0.8, beta=0.25):
    if mode == "translate":
        cx, cy, R = theta, 0.0, R0
    else:  # dilate
        cx, cy, R = 0.0, 0.0, R0 + beta*theta
    return area_int(f, theta, cx, cy, R)

def dL_pred(theta, mode, R0=0.8, beta=0.25):
    if mode == "translate":
        cx, cy, R = theta, 0.0, R0
        vn = lambda phi: np.cos(phi)        # v·n for x-translation
    else:  # dilate
        cx, cy, R = 0.0, 0.0, R0 + beta*theta
        vn = lambda phi: beta + 0*phi       # v·n for uniform radial speed
    A = area_int(dft, theta, cx, cy, R)
    B = bnd_int(theta, cx, cy, R, vn)
    return A + B

def check(theta, mode):
    h = d_theta
    fd   = (L(theta+h, mode) - L(theta-h, mode)) / (2*h)
    pred = dL_pred(theta, mode)
    return {"mode": mode, "theta": theta, "fd": fd, "pred": pred, "abs_err": abs(fd - pred)}

for mode in ["translate", "dilate"]:
    for t in [0.2, 0.7, -0.4]:
        print(check(t, mode))


{'mode': 'translate', 'theta': 0.2, 'fd': 2.376580212605006, 'pred': 2.3766222917196695, 'abs_err': 4.207911466336256e-05}
{'mode': 'translate', 'theta': 0.7, 'fd': -3.2140740725831174, 'pred': -3.214080595480241, 'abs_err': 6.522897123772964e-06}
{'mode': 'translate', 'theta': -0.4, 'fd': -0.09959610822740661, 'pred': -0.0995783485724766, 'abs_err': 1.7759654930005198e-05}
{'mode': 'dilate', 'theta': 0.2, 'fd': 1.563893096194896, 'pred': 1.5726818240624953, 'abs_err': 0.008788727867599322}
{'mode': 'dilate', 'theta': 0.7, 'fd': -2.5573159615828454, 'pred': -2.5593409402212375, 'abs_err': 0.0020249786383921276}
{'mode': 'dilate', 'theta': -0.4, 'fd': -0.11761048091835091, 'pred': -0.11684420401159183, 'abs_err': 0.0007662769067590808}
